# SPEEDWAY GRAND PRIX (SGP) 2015 - 2023 DATA ANALYSIS PROJECT

In [53]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime


### Getting data from gurustats

In [54]:
#t1 - punktacja per runda
#t2 - półfinały

# Get the current year
current_year = datetime.now().year

# Create a list of years from current_year to 2016
years = np.arange(current_year,2014,-1)

# The url prefix and suffix
url_prefix = 'http://gurustats.pl/app/#/sgp'
url_suffix = 'gen'

# Initialize the WebDriver
service = Service(verbose = True)
driver = webdriver.Edge(service = service)

data_t1 = []
#data_t2 = []

try:
    for year in years:
        # Construct the URL for the specific year
        url = f'{url_prefix}{year%100:02d}{url_suffix}'
        
        driver.get(url)
        html = driver.page_source

        # Read the HTML tables and clean column names
        tables = pd.read_html(html)
        tables[0].columns = tables[0].columns.str.replace(r"[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ0-9\.\-\s]+",'',regex=True)

        #Insert the 'ROK' column
        tables[0].insert(tables[0].columns.get_loc("MSC"),'ROK',year)

        #Devide tables[0] for two DataFrames
        t1 = tables[0]
        
        
        
        data_t1.append(t1)
        
finally:
    driver.quit()


# Concatenate the dataframes and transform data
df_t1 = pd.concat(data_t1,axis=0, ignore_index=True)
df_t2 = df_t1.loc[:,['ROK','MSC','ZAWODNIK','PAŃSTWO']+ [f'R{i}' for i in range(1,13)]]
df_t2 = pd.melt(df_t2,id_vars=['ROK','MSC','ZAWODNIK', 'PAŃSTWO'],
                value_vars=[f'R{i}' for i in range(1,13)],
                var_name='RUNDA',
                value_name='PUNKTY PER RUNDA')
#drop null values
df_t2.dropna(subset='PUNKTY PER RUNDA', inplace=True)

df_t1 = df_t1.loc[:,['ROK','MSC','ZAWODNIK','PAŃSTWO','RAZEM','WYGRANE','TOP 3','FINAŁY','PÓŁFINAŁY','SKUT. -FINAŁY', 'SKUT. -PÓŁFINAŁY']]






In [55]:
df_t1

,ROK,MSC,ZAWODNIK,PAŃSTWO,RAZEM,WYGRANE,TOP 3,FINAŁY,PÓŁFINAŁY,SKUT. -FINAŁY,SKUT. -PÓŁFINAŁY
0,2023,1,Bartosz Zmarzlik,Polska,158,5,7,8,9,63%,89%
1,2023,2,Fredrik Lindgren,Szwecja,150,1,6,8,9,13%,89%
2,2023,3,Martin Vaculik,Słowacja,125,2,4,4,7,50%,57%
3,2023,4,Jack Holder,Australia,123,0,5,5,8,00%,63%
4,2023,5,Leon Madsen,Dania,111,1,4,4,6,25%,67%
...,...,...,...,...,...,...,...,...,...,...,...
175,2015,12,Troy Batchelor,Australia,59,0,1,0,1,NaN,00%
176,2015,13,Tomas H. Jonasson,Szwecja,55,0,0,0,0,NaN,NaN
177,2015,14,Chris Harris,Wielka Brytania,55,0,1,0,1,NaN,00%
178,2015,15,Krzysztof Kasprzak,Polska,45,0,0,0,2,NaN,00%


In [56]:
#export data to .csv file
df_t1.to_csv('I:/Projects/SGP/t1_klasyfikacja_generalna_no_round.csv',index=False,decimal=',')

In [57]:
df_t2

,ROK,MSC,ZAWODNIK,PAŃSTWO,RUNDA,PUNKTY PER RUNDA
0,2023,1,Bartosz Zmarzlik,Polska,R1,20.0
1,2023,2,Fredrik Lindgren,Szwecja,R1,16.0
2,2023,3,Martin Vaculik,Słowacja,R1,9.0
3,2023,4,Jack Holder,Australia,R1,8.0
4,2023,5,Leon Madsen,Dania,R1,4.0
...,...,...,...,...,...,...
2154,2015,11,Michael Jepsen Jensen,Dania,R12,7.0
2155,2015,12,Troy Batchelor,Australia,R12,2.0
2156,2015,13,Tomas H. Jonasson,Szwecja,R12,4.0
2157,2015,14,Chris Harris,Wielka Brytania,R12,0.0


In [58]:
df_t2.to_csv('I:/Projects/SGP/t2_zawodnik_rundy_pkt.csv',index=False,decimal=',')

In [62]:
#t3 - statystyki zawodników

# Get the current year
current_year = datetime.now().year

# Create a list of years from current_year to 2016
years = np.arange(current_year,2014,-1)

# Define the base URL
url_base = 'http://gurustats.pl/app/#/sgp'

# Initialize the WebDriver
service = Service(verbose = True)
driver = webdriver.Edge(service = service)


data_t3 = []

try:
    for year in years:
        # Construct the URL for the specific year
        url = f'{url_base}{year%100:02d}ind'

        driver.get(url)
        html = driver.page_source

        # Read the HTML tables and clean column names
        tables = pd.read_html(html)
        
        #Format columns name
        tables[0].columns = tables[0].columns.str.replace(r"[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ0-9\.\-\s\%]+",'',regex=True)

        #Insert the 'ROK' column
        tables[0].insert(tables[0].columns.get_loc("ZAWODNIK"),"ROK",year)

        t3 = tables[0]

        # Divide the 'ŚREDNIA' column by 1000
        t3['ŚREDNIA'] = t3['ŚREDNIA'] / 1000

        # Rename columns
        t3.rename(columns={'3':'1MSC',
                        '2':'2MSC',
                        '1':'3MSC',
                        '0':'4MSC',
                        'D':'DEFEKTY',
                        'W':'WYKLUCZENIA',
                        'U':'UPADKI',
                        'T':'TASMY',
                        'D.1': 'D'}, inplace=True)

        data_t3.append(t3)

finally:
    # Close the WebDriver to release resources
    driver.quit()

# Concatenate the data frames
df_t3 = pd.concat(data_t3,axis=0,ignore_index=True)

#Replacing "Anglia" for "Wielka Brytania"
df_t3.PAŃSTWO = df_t3.PAŃSTWO.str.replace('Anglia','Wielka Brytania')


In [64]:
df_t3

,MSC,ROK,ZAWODNIK,PAŃSTWO,PKT,PKTGP,BIEGI,ŚREDNIA,1MSC,2MSC,...,4MSC,DEFEKTY,WYKLUCZENIA,UPADKI,TASMY,%ZW,A,B,C,D
0,1,2023,Bartosz Zmarzlik,Polska,136.0,158.0,62.0,2.194,32,15,...,5,0,3,0,1,52%,NaN,NaN,NaN,NaN
1,2,2023,Fredrik Lindgren,Szwecja,129.0,150.0,67.0,1.925,23,24,...,8,0,2,0,0,34%,NaN,NaN,NaN,NaN
2,3,2023,Martin Vaculik,Słowacja,107.0,125.0,61.0,1.754,19,19,...,11,1,2,0,0,31%,NaN,NaN,NaN,NaN
3,4,2023,Jack Holder,Australia,111.0,123.0,58.0,1.914,19,19,...,4,0,0,0,0,33%,NaN,NaN,NaN,NaN
4,5,2023,Leon Madsen,Dania,95.0,111.0,60.0,1.583,17,15,...,14,0,0,0,0,28%,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,12,2015,Troy Batchelor,Australia,NaN,NaN,NaN,1.017,8,10,...,25,1,3,0,0,14%,1375.0,615.0,750.0,1176.0
176,13,2015,Tomas H. Jonasson,Szwecja,NaN,NaN,NaN,1.019,3,13,...,18,1,1,1,0,06%,1071.0,1083.0,857.0,1071.0
177,14,2015,Chris Harris,Wielka Brytania,NaN,NaN,NaN,0.932,5,10,...,24,2,1,0,0,08%,1167.0,813.0,938.0,867.0
178,15,2015,Krzysztof Kasprzak,Polska,NaN,NaN,NaN,0.750,8,6,...,37,8,0,0,0,13%,786.0,1000.0,667.0,500.0


In [65]:
df_t3.to_csv('I:/Projects/SGP/t3_statystyki_zawodników.csv',index=False,decimal=',')

In [66]:
#t4 - statystyki reprezentacji

#Get the current year
current_year = datetime.now().year

# Create a list of years from current_year to 2016
years = np.arange(current_year,2014,-1)

# Define the base URL
url_base = 'http://gurustats.pl/app/#/sgp'

# Initialize the WebDriver
service = Service(verbose = True)
driver = webdriver.Edge(service = service)


data_t4 = []

try:
    for year in years:
        # Construct the URL for the specific year
        url = f'{url_base}{year%100:02d}pan'

        driver.get(url)
        html = driver.page_source

        # Read the HTML tables and clean column names
        tables = pd.read_html(html)
        
        #Format columns name
        tables[0].columns = tables[0].columns.str.replace(r"[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ0-9\.\-\s\%]+",'',regex=True)

        #Insert the 'ROK' column
        tables[0].insert(tables[0].columns.get_loc("PAŃSTWO"),"ROK",year)

        t4 = tables[0]

        # Divide the 'ŚREDNIA' column by 1000
        t4['ŚREDNIA'] = t4['ŚREDNIA'] / 1000

        # Rename columns
        t4.rename(columns={'3': '1MSC', '2': '2MSC', '1': '3MSC', '0': '4MSC'}, inplace=True)


        data_t4.append(t4)

finally:
    # Close the WebDriver to release resources
    driver.quit()

# Concatenate the data frames
df_t4 = pd.concat(data_t4,axis=0,ignore_index=True)

# Replacing "Anglia" for "Wielka Brytania"
df_t4 = df_t4.drop(df_t4[df_t4.PAŃSTWO.str.contains('Anglia')].index)


In [67]:
df_t4

,ROK,PAŃSTWO,PKT,BIEGI,ŚREDNIA,1MSC,2MSC,3MSC,4MSC,%ZW,WYGRANE,TOP 3,FINAŁY,PÓŁFINAŁY
0,2023,Australia,284,170,1.671,48,46,28,48,28%,NaN,6.0,9.0,19.0
1,2023,Polska,269,174,1.546,46,45,42,41,26%,5.0,8.0,10.0,14.0
2,2023,Wielka Brytania,267,168,1.589,54,35,44,35,32%,1.0,2.0,4.0,19.0
3,2023,Dania,215,158,1.361,32,41,48,37,20%,1.0,4.0,4.0,10.0
4,2023,Szwecja,179,121,1.479,26,33,27,35,21%,1.0,6.0,9.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2015,USA,147,71,2.070,35,15,12,9,49%,2.0,4.0,4.0,9.0
91,2015,Słowenia,109,72,1.514,16,20,21,15,22%,2.0,3.0,3.0,5.0
92,2015,Łotwa,3,5,0.600,0,1,1,3,00%,0.0,0.0,0.0,0.0
93,2015,Finlandia,3,7,0.429,0,0,3,4,00%,0.0,0.0,0.0,0.0


In [68]:
df_t4.to_csv('I:/Projects/SGP/t4_statystyki_reprezentacji.csv',index=False,decimal=',')

In [69]:
#t5 - statystyki torów

#get the current year
current_year = datetime.now().year

# Create a list of years from current_year to 2016
years = np.arange(current_year,2014,-1)

# Define the base URL
url_base = 'http://gurustats.pl/app/#/sgp'

# Initialize the WebDriver
service = Service(verbose = True)
driver = webdriver.Edge(service = service)



data_t5 = []

try:
    for year in years:
        # Construct the URL for the specific year
        url = f'{url_base}{year%100:02d}tory'

        driver.get(url)
        html = driver.page_source

        # Read the HTML tables and clean column names
        tables = pd.read_html(html)

        #Format columns name
        tables[0].columns = tables[0].columns.str.replace(r"[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ0-9\.\-\s\%]+",'',regex=True)

        #Insert the 'ROK' column
        tables[0].insert(tables[0].columns.get_loc("RUNDA"),"ROK",year)

        t5 = tables[0]

        data_t5.append(t5)

finally:
    # Close the WebDriver to release resources
    driver.quit()

# Concatenate the data frames
df_t5 = pd.concat(data_t5,axis=0,ignore_index=True)

In [70]:
df_t5

,ROK,RUNDA,TOR,ZWTOR A,ZWTOR B,ZWTOR C,ZWTOR D,%ZWTOR A,%ZWTOR B,%ZWTOR C,%ZWTOR D,PKTTOR A,PKTTOR B,PKTTOR C,PKTTOR D,%PKTTOR A,%PKTTOR B,%PKTTOR C,%PKTTOR D
0,2023,1,Gorican,10,4,0,9,43%,17%,00%,39%,40,38,16,44,29%,28%,12%,32%
1,2023,2,Warszawa,6,8,2,7,26%,35%,09%,30%,37,35,26,40,27%,25%,19%,29%
2,2023,3,Praga,7,7,4,5,30%,30%,17%,22%,41,43,24,30,30%,31%,17%,22%
3,2023,4,Teterow,14,1,0,8,61%,04%,00%,35%,57,24,19,38,41%,17%,14%,28%
4,2023,5,Gorzów,8,5,3,7,35%,22%,13%,30%,43,33,24,38,31%,24%,17%,28%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2015,8,Gorzów,8,5,6,4,35%,22%,26%,17%,37,34,39,28,27%,25%,28%,20%
90,2015,9,Krsko,9,8,2,4,39%,35%,09%,17%,45,38,28,27,33%,28%,20%,20%
91,2015,10,Sztokholm,10,7,2,4,43%,30%,09%,17%,48,36,21,33,35%,26%,15%,24%
92,2015,11,Toruń,4,9,5,5,17%,39%,22%,22%,26,42,33,37,19%,30%,24%,27%


In [71]:
df_t5.to_csv('I:/Projects/SGP/t5_statystyki_torow.csv',index=False,decimal=',')

In [72]:
#t6 - round details
from datetime import datetime
from calendar import month_abbr

# Get the current year
current_year = datetime.now().year

# Create a range of years from the current year to 2015
years = np.arange(current_year,2014,-1)

# Define the URL prefix and suffix
url_prefix = 'https://en.wikipedia.org/wiki/'
url_suffix = '_Speedway_Grand_Prix'

data_t6 = []

for year in years:
    # Read the table at index 3 from the URL
    t6 = pd.read_html(f'{url_prefix}{year}{url_suffix}')[3]

    # Insert the 'Year' column
    t6.insert(t6.columns.get_loc("Round"),"Year",year)

    data_t6.append(t6)

# Concatenate the data
df_t6 = pd.concat(data_t6,axis=0,ignore_index=True)

#df_t6.drop(columns='Results',inplace=True)

# Split the 'Date' column into 'Month' and 'Day'
df_t6['Day'] = df_t6['Date'].str.extract(r'(\d+)')
df_t6['Month'] = df_t6['Date'].str.extract(r'([A-Za-z]+)')


# Mapping month names to numbers
lower_ma = [m.lower() for m in month_abbr]
df_t6['Month_abr'] = df_t6.Month.str.lower().str[:3].map(lambda m: lower_ma.index(m)).astype('Int8')

# Create the 'Full_Date' column
df_t6['Full_Date_Str'] = df_t6['Year'].astype(str) + '-' + df_t6['Month_abr'].astype(str) + '-' + df_t6['Day'].astype(str)
df_t6['Round Date'] = pd.to_datetime(df_t6['Full_Date_Str'], format='%Y-%m-%d', errors='raise')

# Drop the 'Results' and 'Full_Date_Str column
df_t6.drop(columns=['Full_Date_Str','Results'],inplace=True)

In [73]:
df_t6

,Year,Round,Date,City and venue,Winner,Runner-up,3rd placed,4th placed,Day,Month,Month_abr,Round Date
0,2023,1,29 April,"Goričan, Croatia Stadium Milenium",Bartosz Zmarzlik,Robert Lambert,Fredrik Lindgren,Jason Doyle,29,April,4,2023-04-29
1,2023,2,13 May,"Warsaw, Poland Stadion Narodowy",Fredrik Lindgren,Jack Holder,Bartosz Zmarzlik,Jason Doyle,13,May,5,2023-05-13
2,2023,3,3 June,"Prague, Czech Republic Markéta Stadium",Martin Vaculík,Leon Madsen,Jack Holder,Bartosz Zmarzlik,3,June,6,2023-06-03
3,2023,4,10 June,"Teterow, Germany Bergring Arena",Bartosz Zmarzlik,Jason Doyle,Jack Holder,Kim Nilsson,10,June,6,2023-06-10
4,2023,5,24 June,"Gorzów, Poland Edward Jancarz Stadium",Bartosz Zmarzlik,Leon Madsen,Fredrik Lindgren,Jason Doyle,24,June,6,2023-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...
89,2015,8,August 29,"Gorzów , Poland Edward Jancarz Stadium",Matej Žagar,Greg Hancock,Tai Woffinden,Bartosz Zmarzlik,29,August,8,2015-08-29
90,2015,9,September 12,"Krško , Slovenia Matija Gubec Stadium",Greg Hancock,Tai Woffinden,Peter Kildemand,Nicki Pedersen,12,September,9,2015-09-12
91,2015,10,September 26,"Stockholm , Sweden Friends Arena",Tai Woffinden,Greg Hancock,Niels-Kristian Iversen,Maciej Janowski,26,September,9,2015-09-26
92,2015,11,October 3,"Toruń , Poland Rose Motoarena",Nicki Pedersen,Jason Doyle,Maciej Janowski,Niels-Kristian Iversen,3,October,10,2015-10-03


In [388]:
df_t6.to_csv('I:/Projects/SGP/t6_rundy_detale.csv',index=False,decimal=',')